In [2]:
import pandas as pd
import pandas_profiling as pdp
import lightgbm as lgb
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
import optuna

In [3]:
train = pd.read_csv('../processed_data/train_v11.csv')
test = pd.read_csv('../processed_data/test_v11.csv')

In [9]:
test['prob']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
31257    0.0
31258    0.0
31259    0.0
31260    0.0
31261    0.0
Name: prob, Length: 31262, dtype: float64

In [14]:
train['y'][:len(test)] * (1-test['prob'])

0         75000.0
1         76000.0
2        110000.0
3        150000.0
4         74000.0
           ...   
31257     87500.0
31258    170000.0
31259    275000.0
31260    158000.0
31261     62000.0
Length: 31262, dtype: float64